In [1]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 9.4 MB/s eta 0:00:00:00:0100:01


# Get Dataset

In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="cJUunyRGd3YuAM3cnXbU")
project = rf.workspace("volleyunder").project("vv_dataset_only_ball")
version = project.version(2)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...


In [7]:
dataset.location

'/home/khalil/VAS/training/vv_dataset_only_ball-2'

In [ ]:
import shutil

shutil.move('vv_dataset_only_ball-2/train',
            'vv_dataset_only_ball-2/vv_dataset_only_ball-2/train'
            )

shutil.move('vv_dataset_only_ball-2/test',
            'vv_dataset_only_ball-2/vv_dataset_only_ball-2/test'
            )

shutil.move('vv_dataset_only_ball-2/valid',
            'vv_dataset_only_ball-2/vv_dataset_only_ball-2/valid'
            )

# Training

In [9]:
#cleaning cash from gpu

import torch
torch.cuda.empty_cache()


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml epochs=20 imgsz=420 batch=8 amp=False

New https://pypi.org/project/ultralytics/8.3.40 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 3721MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/khalil/VAS/training/vv_dataset_only_ball-2/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=420, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

# Evaluation

In [10]:
from ultralytics import YOLO

In [ ]:
# Load the trained model
model = YOLO('runs/detect/train12/weights/best.pt')

# Validate the model on the validation set
metrics = model.val(data='/home/khalil/VAS/training/vv_dataset_only_ball-2/data.yaml')
print(metrics)  

# Inference

In [18]:
import cv2
from matplotlib import pyplot as plt

# Run YOLO inference
results = model('../image1.jpg')  
print(results[0].boxes)


image 1/1 /home/khalil/VAS/training/../image1.jpg: 320x448 1 ball, 36.9ms
Speed: 15.8ms preprocess, 36.9ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 448)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.3840], device='cuda:0')
data: tensor([[321.4642, 159.8530, 327.0332, 168.0201,   0.3840,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (333, 500)
shape: torch.Size([1, 6])
xywh: tensor([[324.2487, 163.9365,   5.5691,   8.1671]], device='cuda:0')
xywhn: tensor([[0.6485, 0.4923, 0.0111, 0.0245]], device='cuda:0')
xyxy: tensor([[321.4642, 159.8530, 327.0332, 168.0201]], device='cuda:0')
xyxyn: tensor([[0.6429, 0.4800, 0.6541, 0.5046]], device='cuda:0')


In [21]:
import cv2
from matplotlib import pyplot as plt

image_path = '../image1.jpg'

# Run YOLO inference
results = model(image_path)
print(results[0].boxes.data)

# Extract the first bounding box details
bbox_data = results[0].boxes.data[0].cpu().numpy()  # Convert tensor to numpy
x_min, y_min, x_max, y_max, confidence, class_id = bbox_data

# Load the image

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Draw the bounding box
x_min, y_min, x_max, y_max = map(int, [x_min, y_min, x_max, y_max])
label = f"Ball {confidence:.0%}"
color = (255, 255, 0)  # Yellow color in RGB

# Draw rectangle
cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 1)

# Add label
font_scale = 1
font_thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
text_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
text_w, text_h = text_size
cv2.rectangle(image, (x_min, y_min - text_h - 10), (x_min + text_w, y_min), color, -1)
cv2.putText(image, label, (x_min, y_min - 5), font, font_scale, (0, 0, 0), font_thickness)

# Save the output image
output_path = '../bbox_output.png'
cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Show the image
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.show()

print(f"Image saved as {output_path}")



image 1/1 /home/khalil/VAS/training/../image1.jpg: 320x448 1 ball, 19.7ms
Speed: 1.4ms preprocess, 19.7ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 448)
tensor([[321.4642, 159.8530, 327.0332, 168.0201,   0.3840,   0.0000]], device='cuda:0')


<Figure size 1000x800 with 1 Axes>

Image saved as ../bbox_output.png
